<a href="https://colab.research.google.com/github/the-hornery/colab-notebooks/blob/main/Photobait_Forgewebui_Optimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
import shutil
from google.colab import drive

# Configuration
forge_dir = "/content/forge"
output_dir = "/content/drive/MyDrive/Output"
lora_drive_dir = "/content/drive/MyDrive/Lora"
forge_lora_dir = os.path.join(forge_dir, "models", "Lora", "user")
os.environ["HF_HUB_ENABLE_HF_TRANSFER"]="1"
os.environ["HF_HOME"]="/content/drive/MyDrive/.cache/huggingface"
os.environ["HUGGINGFACE_HUB_CACHE"]=os.environ["HF_HOME"]

print("🔧 Installing dependencies...")
!apt -y update -qq && apt -y install -qq git wget libgl1 libglib2.0-0

print("\n📁 Cloning forge webui...")
%cd /content/
if not os.path.exists(forge_dir):
    !git clone https://github.com/lllyasviel/stable-diffusion-webui-forge forge
else:
    print("✅ Repo already cloned.")

print("\n🔗 Mounting Google Drive...")
drive.mount('/content/drive', force_remount=True)
os.makedirs(output_dir, exist_ok=True)

# Clean out conflicting wheels
!pip uninstall -y xformers torch torchvision torchaudio protobuf
# Install a deterministic, Colab-friendly stack
!pip install --index-url https://download.pytorch.org/whl/cu121 \
  "torch==2.3.1+cu121" "torchvision==0.18.1+cu121"

!pip install --no-deps "xformers==0.0.27"  # works with Torch 2.3.1/cu121 on T4
# (optional) a conservative bnb for older CUDA toolkits:
# pip install "bitsandbytes==0.42.0"
!pip install "insightface==0.7.3"
# (optional) set allocator to reduce fragmentation on T4
import os, torch
os.environ["PYTORCH_CUDA_ALLOC_CONF"]="max_split_size_mb:128"
print("Torch:", torch.__version__, "CUDA:", torch.version.cuda)

🔧 Installing dependencies...
39 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
libgl1 is already the newest version (1.4.0-1).
git is already the newest version (1:2.34.1-1ubuntu1.15).
libglib2.0-0 is already the newest version (2.72.4-0ubuntu2.6).
wget is already the newest version (1.21.2-2ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.

📁 Cloning forge webui...
/content
✅ Repo already cloned.

🔗 Mounting Google Drive...
Mounted at /content/drive
Found existing installation: xformers 0.0.27
Uninstalling xformers-0.0.27:
  Successfully uninstalled xformers-0.0.27
Found existing installation: torch 2.3.1+cu121
Uninstalling torch-2.3.1+cu121:
  Successfully uninstalled torch-2.3.1+cu121
Found existing installation: torchvision 0.18.1+cu121
Uninstalling t

Torch: 2.3.1+cu121 CUDA: 12.1


In [ ]:
import torch, os
print("Torch:", torch.__version__, "CUDA:", torch.version.cuda)
try:
    import xformers, xformers.ops as xops
    print("xFormers loaded:", hasattr(xops, "memory_efficient_attention"))
except Exception as e:
    print("xFormers not loaded:", e)

import os, shutil, pathlib

forge_dir = "/content/forge"
local_output = "/content/outputs"
forge_output = os.path.join(forge_dir, "outputs")
drive_output = "/content/drive/MyDrive/Output"

pathlib.Path(local_output).mkdir(parents=True, exist_ok=True)
# point Forge outputs to local
if os.path.islink(forge_output): os.unlink(forge_output)
elif os.path.isdir(forge_output): shutil.rmtree(forge_output)
os.symlink(local_output, forge_output)
print("Forge will save to:", local_output)

Torch: 2.3.1+cu121 CUDA: 12.1
xFormers loaded: True
Forge will save to: /content/outputs


In [ ]:
from huggingface_hub import hf_hub_download
import torch; torch.set_float32_matmul_precision("high")

os.environ["PYTORCH_CUDA_ALLOC_CONF"]="max_split_size_mb:128"


flux1_repo_id = "lllyasviel/flux1-dev-bnb-nf4"
flux1_filename = "flux1-dev-bnb-nf4-v2.safetensors"
flux1_dest_path = os.path.join(forge_dir, "models", "Stable-diffusion", "flux1-dev-bnb-nf4", flux1_filename)

if not os.path.exists(flux1_dest_path):
    print("\n🧠 Downloading flux1-dev-bnb-nf4...")
    model_file = hf_hub_download(repo_id=flux1_repo_id, filename=flux1_filename, cache_dir="/content/huggingface_cache")
    os.makedirs(os.path.dirname(flux1_dest_path), exist_ok=True)
    shutil.copy(model_file, flux1_dest_path)
    print(f"✅ Model downloaded to : {flux1_dest_path}")
else:
    print(f"✅ Model present in : {flux1_dest_path}")

# Organise outputs
forge_output = os.path.join(forge_dir, "outputs")
if os.path.exists(forge_output) or os.path.islink(forge_output):
    if os.path.islink(forge_output):
        os.unlink(forge_output)
    else:
        shutil.rmtree(forge_output)
os.symlink(output_dir, forge_output)
print(f"📂 Outputs stored in : {output_dir}")

# Links for LoRAs
os.makedirs(os.path.join(forge_dir, "models", "Lora"), exist_ok=True)
if os.path.exists(forge_lora_dir) or os.path.islink(forge_lora_dir):
    if os.path.islink(forge_lora_dir):
        os.unlink(forge_lora_dir)
    else:
        shutil.rmtree(forge_lora_dir)

os.symlink(lora_drive_dir, forge_lora_dir)
print(f"✅ Symlink for LoRA is : {forge_lora_dir} → {lora_drive_dir}")


🧠 Downloading flux1-dev-bnb-nf4...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


flux1-dev-bnb-nf4-v2.safetensors:   0%|          | 0.00/12.0G [00:00<?, ?B/s]

✅ Model downloaded to : /content/forge/models/Stable-diffusion/flux1-dev-bnb-nf4/flux1-dev-bnb-nf4-v2.safetensors
📂 Outputs stored in : /content/drive/MyDrive/Output
✅ Symlink for LoRA is : /content/forge/models/Lora/user → /content/drive/MyDrive/Lora


In [ ]:

import json

ui_config_path = os.path.join(forge_dir, "ui-config.json")
ui_settings = {
    "txt2img/Steps": 28,
    "txt2img/Width": 768,
    "txt2img/Height": 1024,
    "txt2img/Script name": "adetailer"
}

# MAJ ui-config
try:
    current_config = {}
    if os.path.exists(ui_config_path):
        with open(ui_config_path, "r") as f:
            try:
                current_config = json.load(f)
            except json.JSONDecodeError:
                print("⚠️ JSON error - new file needed.")
    current_config.update(ui_settings)
    with open(ui_config_path, "w") as f:
        json.dump(current_config, f, indent=4)
    print(f"✅ ui-config.json saved.")
except Exception as e:
    print(f"❌ UI Config error : {e}")

# Extension Adetailer
adetailer_extension_dir = os.path.join(forge_dir, "extensions", "adetailer")
if not os.path.exists(adetailer_extension_dir):
    !git clone https://github.com/Bing-su/adetailer.git {adetailer_extension_dir}
    print("✅  Adetailer extension installed.")
else:
    print("✅ Adetailer extension is already present.")


✅ ui-config.json saved.
Cloning into '/content/forge/extensions/adetailer'...
remote: Enumerating objects: 3087, done.
remote: Counting objects: 100% (977/977), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 3087 (delta 921), reused 854 (delta 853), pack-reused 2110 (from 3)
Receiving objects: 100% (3087/3087), 572.49 KiB | 2.92 MiB/s, done.
Resolving deltas: 100% (1799/1799), done.
✅  Adetailer extension installed.


In [ ]:

%cd {forge_dir}

print("🚀 Launching Forge (Gradio)...")
!python launch.py \
  --share \
  --ckpt-dir models/Stable-diffusion/flux1-dev-bnb-nf4 \
  --xformers \
  --opt-sdp-attention

/content/forge
🚀 Launching Forge (Gradio)...
INCOMPATIBLE PYTHON VERSION

This program is tested with 3.10.6 Python, but you have 3.12.11.
If you encounter an error with "RuntimeError: Couldn't install torch." message,
or any other error regarding unsuccessful package (library) installation,
please downgrade (or upgrade) to the latest version of 3.10 Python
and delete current Python and "venv" folder in WebUI's directory.

You can download 3.10 Python from here: https://www.python.org/downloads/release/python-3106/



Use --skip-python-version-check to suppress this warning.
Python 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Version: f2.0.1v1.10.1-previous-669-gdfdcbab6
Commit hash: dfdcbab685e57677014f05a3309b48cc87383167
Installing clip
Installing open_clip
Cloning assets into /content/forge/repositories/stable-diffusion-webui-assets...
Cloning into '/content/forge/repositories/stable-diffusion-webui-assets'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 1

In [ ]:
!ls -lat /content/forge/outputs | head
!ls -lat /content/forge/outputs/txt2img-images | head